In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error

E:\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [4]:
def savemodel(model,name):
    path=r"C:\Users\73454\Desktop\NCBDC 2019\model"+'\\'+name+'.pkl'
    joblib.dump(model,path)
def loadmodel(name):
    path=r"C:\Users\73454\Desktop\NCBDC 2019\model"+'\\'+name+'.pkl'
    return joblib.load(path)

In [5]:
#首先加载数据  #本来需要加载所有数据进行归一化的，但是这里为了简化代码，直接读取归一化输入的训练数据和测试数据
trainlist,testlist=[],[]
alldata=pd.DataFrame()
TestLength=7
TrainLength=30
for i in range(TrainLength):
    df=pd.read_csv(r"C:\Users\73454\Desktop\NCBDC 2019\SmallSizeTrainData"+'\\'+str(i)+".csv").iloc[:,1:]
    trainlist.append(df)
    alldata=pd.concat([alldata,df])
for i in range(TestLength):
    df=pd.read_csv(r"C:\Users\73454\Desktop\NCBDC 2019\SmallSizeTestData"+'\\'+str(i)+".csv").iloc[:,1:]
    testlist.append(df)

# Single step model

In [29]:
#前面数据都加载好了，现在决定一下我们的模型的输入输出
print("我们有这么多的特征：",trainlist[0].columns)
#我们将可以输入的特征分类，分成低频特征和高频特征
#低频特征： 'soc', 'temp_max', 'temp_min','tempMAXMINdiff','SOCgap'
#高频特征：'speed', 'total_voltage', 'total_current', 'motor_voltage', 'motor_current', 'total_P', 'motor_P'
#这里的speed有待争议，如果之前的速度有在模型中回归出来，那么可以放到之后时间段作为特征

#这里的输出也可以有各种输出，比如timestep范围内的里程增量[milediff].sum()
#或者回归最后一个数据距离一开始的里程的里程增量milegap
#还可以加上速度speed，或者速度增量speeddiff

#这里输出和输入的变化以及模型的调参就交给晓丹，我们这里以一个简单的为例子
InputFeatures=['SOCgap','soc', 'total_voltage', 'total_current', 'motor_voltage', 'motor_current', 'total_P', 'motor_P']
OutputFeatures=['milediff']
TimeStep=36   #120*10/60=20min（）
AvrgTime=1

我们有这么多的特征： Index(['total_voltage', 'total_current', 'soc', 'temp_max', 'temp_min',
       'motor_voltage', 'motor_current', 'total_P', 'motor_P',
       'tempMAXMINdiff', 'SOCgap', 'speed', 'mileage', 'milediff', 'speeddiff',
       'milegap'],
      dtype='object')


In [30]:
#这个功能主要是把一个df转换成 timestep为一组的df的list 注意 有一些数据集其数据个数<=timestep 会导致输出为[]
def df2Timedf(df,inputFeature,outputFeature, timestep):
    end_index=len(df)
    data=[]
    for i in range(timestep,end_index):
        indices=range(i-timestep,i)
        data.append(df.loc[indices,inputFeature+outputFeature])
    if data!=[]:#数据集其数据个数<=timestep 会导致输出为[] 传递None
        return data
    else:
        return 
#此处是将输入改成（个数，timestep,features）的array,输出（可以改的）是20mins的milediff的和
# 可以满足RNN的需要了
def Timedf2RNNdata(Timedf,inputFeature,outputFeature):#此函数未处理None Timedf情况
    Xset,Yset=[],[]
    for df in Timedf:
        X=df.loc[:,inputFeature].to_numpy()
        Y=df.loc[:,outputFeature].to_numpy()
        Xset.append(X)
        Yset.append(Y.sum())#注意此处.sum()
    Xset=np.array(Xset)
    Yset=np.array(Yset)
    Yset=np.reshape(Yset,(-1,))
    return Xset,Yset
#下面是我们的机器学习方法的数据转换，需要将（None，120，8）的数据变成（None，10*8）的数据
#也是就说我们每12个数据就需要变成一个数据（取平均）avrgTime=12 (120/12=10)
def Timedf2GDBT(Timedf,inputFeature,outputFeatures,avrgTime):
    if(Timedf==None):
        print("Timedf contain nothing")
        return None,None   #如果timedf为None 输出数据也是None，None
    Xset,Yset=[],np.array([])
    length=len(Timedf[0])
    for df in Timedf:
        X=df.loc[:,inputFeature]
        Y=df.loc[:,outputFeatures]
        Xc=np.array([])
        for i in range(int(length/avrgTime)):#这里就是 range(10)
            begin_index=i*avrgTime
            end_index=(i+1)*avrgTime
            dataX=np.array(X[begin_index:end_index].mean())
            Xc=np.append(Xc,dataX)
        Xset.append(Xc)
        Yset=np.append(Yset,Y.sum()[0])
    Xset=np.array(Xset)
    return Xset,Yset

这个是读取一个训练集的代码

这个是读取所有训练集的代码

In [31]:
#上面我们定义过 TimeStep=120
SIZE=int(TimeStep/AvrgTime*len(InputFeatures))
trainX,trainY=np.empty((1,SIZE)),np.array([])
for index,df in enumerate(trainlist):
    if(len(df)<=TimeStep):
        continue
    Timedf=df2Timedf(df,InputFeatures,OutputFeatures,TimeStep)
    Xset,Yset=Timedf2GDBT(Timedf,InputFeatures,OutputFeatures,avrgTime=AvrgTime)
    trainX=np.vstack((trainX,Xset))
    trainY=np.append(trainY,Yset)
    print(index,"dataset is already")

0 dataset is already
1 dataset is already
2 dataset is already
3 dataset is already
4 dataset is already
5 dataset is already
6 dataset is already
7 dataset is already
8 dataset is already
9 dataset is already
10 dataset is already
11 dataset is already
12 dataset is already
13 dataset is already
14 dataset is already
15 dataset is already
16 dataset is already
17 dataset is already
18 dataset is already
19 dataset is already
20 dataset is already
21 dataset is already
22 dataset is already
23 dataset is already
24 dataset is already
25 dataset is already
26 dataset is already
27 dataset is already
28 dataset is already
29 dataset is already


In [47]:
trainX=trainX[1:]

In [48]:
def MyModel(Normalized_X,Normalized_Y,spilt_size=0.1,random_key=0,n_estimators=500,max_depth=4):
    ###############################################################################
    # Load data
    x_train, x_test, y_train, y_test = train_test_split(Normalized_X,Normalized_Y,test_size=spilt_size, random_state=random_key)
    ###############################################################################
    # Fit regression model
    params = {'n_estimators': n_estimators, 'max_depth': max_depth, 'min_samples_split': 2,
              'learning_rate': 0.01, 'loss': 'ls'}
    clf = ensemble.GradientBoostingRegressor(**params)
    clf.fit(x_train, y_train)
    y_predict  = clf.predict(x_test)
    mse = mean_squared_error(y_test, y_predict)
    print("MSE: %.4f" % mse)
    return clf

In [52]:
model=MyModel(trainX,trainY,n_estimators=500,max_depth=4)

ValueError: Found input variables with inconsistent numbers of samples: [59141, 59143]

In [27]:
model

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=4, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=500, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

测试方法：先将测试集数据转换成输入，然后得到测试集输出，和测试集真实里程差作比较（最后的里程差）

# 无重叠结果测试

In [12]:
Result = 0
for index,df in enumerate(testlist):
    Timedf=df2Timedf(df,InputFeatures,OutputFeatures,TimeStep)
    Xset,Yset=Timedf2GDBT(Timedf,InputFeatures,OutputFeatures,avrgTime=AvrgTime)
    result=0
    for Index,x in enumerate(Xset):
        if(Index%TimeStep==0):
            a=model.predict(x.reshape(-1, SIZE))[0]
            result=result+a
    print(result,df.iloc[-1].milegap)
    result=(df.iloc[-1].milegap-result)**2
    print(index,result)
    Result=Result+result
Result=Result**(1/2)

79.65163144204081 83.79999999999562
0 17.208961692628076
111.13816113955897 114.39999999999418
1 10.639592751445235
124.88431366212289 121.0
2 15.087892625754554
85.12547915044934 96.59999999999854
3 131.66462872673947
114.49931498739954 117.80000000000292
4 10.894521552424582
132.20219728916376 121.30000000000292
5 118.85790573178593
116.72029213821445 115.5
6 1.4891129025879863


In [13]:
Result

17.488356583263215

## 重叠结果测试

In [14]:
Result2 = 0
for index,df in enumerate(testlist):
    Timedf=df2Timedf(df,InputFeatures,OutputFeatures,TimeStep)
    Xset,Yset=Timedf2GDBT(Timedf,InputFeatures,OutputFeatures,avrgTime=AvrgTime)
    result=0
    for Index,x in enumerate(Xset):
        a=model.predict(x.reshape(-1, SIZE))[0]
        result=result+a
    result=result/TimeStep
    print(result,df.iloc[-1].milegap)
    result=(df.iloc[-1].milegap-result)**2
    print(index,result)
    Result2=Result2+result
Result2=Result2**(1/2)

77.94513072184273 83.79999999999562
0 34.279494264258545
109.07676156051643 114.39999999999418
1 28.336867483533492
125.83786905058209 121.0
2 23.40497695058001
85.11476600934704 96.59999999999854
3 131.91059982001678
111.95320857820772 117.80000000000292
4 34.18496992997797
131.82246749070165 121.30000000000292
5 110.72232209281164
118.5548830211754 115.5
6 9.332310273065737


In [15]:
Result2

19.29174799789392

In [16]:
df=testlist[0]
len(df)

1683

In [17]:
def padding(df):
    lens=len(df)
    print(lens)
    foredata=df.loc[0]
    foredata.total_current=-0.292503883
    foredata.motor_current=-0.3233341680
    foredata.speed=0
    foredf=pd.DataFrame(columns=df.columns)
    for number in range(0,TimeStep):
        foredf.loc[number]=foredata
    backdf=pd.DataFrame(columns=df.columns)
    backdata=df.iloc[lens-1]
    backdata.total_current=-0.292503883
    backdata.motor_current=-0.3233341680
    backdata.speed=0
    for number in range(0,TimeStep):
        backdf.loc[number]=backdata
    mydf=pd.DataFrame(columns=df.columns)
    return pd.concat([foredf,df,backdf],ignore_index=True)

In [18]:
Result3=0
for index,Df in enumerate(testlist):
    df=padding(Df)
    Timedf=df2Timedf(df,InputFeatures,OutputFeatures,TimeStep)
    Xset,Yset=Timedf2GDBT(Timedf,InputFeatures,OutputFeatures,avrgTime=AvrgTime)
    result=0
    for Index,x in enumerate(Xset):
        a=model.predict(x.reshape(-1, SIZE))[0]
        result=result+a
    result=result/TimeStep
    print(result,df.iloc[-1].milegap)
    result=(df.iloc[-1].milegap-result)**2
    print(index,result)
    Result3=Result3+result
Result3=Result3**(1/2)

1683
81.26179565178364 83.79999999999562
0 6.442481313282187
2681
110.10780309609486 114.39999999999418
1 18.422954261842943
2857
127.62805947314246 121.0
2 43.93117237951347
1556
85.93983069594968 96.59999999999854
3 113.63920959098564
2704
113.99672230947188 117.80000000000292
4 14.464921191291175
2685
133.37990692775668 121.30000000000292
5 145.92415138319313
1759
120.37308637897415 115.5
6 23.7469708569434


In [19]:
Result3 #22.9730

19.14606646225412

In [20]:
duelist=[]
DueLength=100
for i in range(DueLength):
    df=pd.read_csv(r"C:\Users\73454\Desktop\NCBDC 2019\NormalizedDueTestData"+'\\'+str(i)+".csv").iloc[:,1:]
    duelist.append(df)

In [29]:
ResultList=[]
for index,df in enumerate(duelist[91:]):
    Timedf=df2Timedf(df,InputFeatures,OutputFeatures,TimeStep)
    Xset,Yset=Timedf2GDBT(Timedf,InputFeatures,OutputFeatures,avrgTime=AvrgTime)
    result=0
    for Index,x in enumerate(Xset):
        if(Index%TimeStep==0):
            a=model.predict(x.reshape(-1, SIZE))[0]
            result=result+a
    print(index,result)
    ResultList.append(result)

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  import sys


0 60.80752313061299
1 83.2567004033096
2 44.482775931923676
3 82.06783160677098
4 26.69214797109212
5 70.21281861687346
6 91.58300463495085
7 98.45853586254395
8 97.40102124508672


In [25]:
rs=pd.DataFrame(ResultList,columns=['milegap'])

In [27]:
rs.to_csv(r"C:\Users\73454\Desktop\NCBDC 2019\Result\Submit2.csv")

In [34]:
savemodel(model,"Day2")
model=loadmodel("Day2")

## 下面是之前写的代码